## Trying RAG From MD Files

In this notebook I will try and extract context for summarisation out of series of .md files. The idea is to create a working ChatBot on initially Streameye's Blog articles, but later Streameye Documentation once it is ready.

Challenges are:

- Vectorising the md files and storing them in a vector DB
- Extracting proper context for user queries
- Using an LLM to answer these queries from the stored data
- Avoiding LLM hallucinations, keeping the answers concise on the supplied context

#### Resources

Great resource by Venelin Valkov [here](https://www.mlexpert.io/prompt-engineering/langchain-quickstart-with-llama-2)

Youtube tutorial and resource that explain the process with OpenAI's embeddings instead of HF [here](https://www.youtube.com/watch?v=tcqEUSNCn8I)

### Preprocessing

#### Loading the md files and tokenising

Its good idea to split big documents or long texts into chunks. The idea is to load them faster and to make each chunk more focused and relevant.


In [1]:
# Installs - need to restart notebook after installs!

!pip install unstructured
!pip install sentence-transformers
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.2 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 41.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 2.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.9/457.9 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 69.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 67.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 8.5 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=5fb3417737cc697b10f6eb5c4ff8ba9ed9c82b

In [2]:
!pip install "unstructured[md]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.9/103.9 kB 803.2 kB/s eta 0:00:000:010:010m


In [3]:
# Includes
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.vectorstores.chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.evaluation import load_evaluator
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains import RetrievalQA
from langchain import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory

from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

import os
import shutil

In [4]:
DATA_PATH = "./data/streameye_blog/"
CHROMA_PATH = "./data/streameye_blog/db"

To split text into chunks we use the **RecursieveCharacterTextSplitter**

In [5]:
def load_documents():
    loader = DirectoryLoader(DATA_PATH, glob="*.md")
    documents = loader.load()
    return documents
def split_text(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")
    return chunks


In [8]:
def save_to_chroma(chunks: list[Document]):
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH) ## Recursive delete
    db = Chroma.from_documents(chunks, HuggingFaceEmbeddings(), persist_directory=CHROMA_PATH)
    db.persist()

In [5]:
docs = load_documents()

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [6]:
chunks = split_text(docs)

Split 37 documents into 569 chunks.


In [9]:
save_to_chroma(chunks)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

This is pretty bad running with the default HF Embeddings. Lets try and put a better LLM to be used. Lets try with a relatively small one first as I dont know if RAM is sufficient.

In [11]:
hf_evaluator = load_evaluator("pairwise_embedding_distance", embeddings=HuggingFaceEmbeddings())
hf_evaluator.evaluate_string_pairs(prediction="apple", prediction_b="orange")

{'score': 0.5988499147809145}

The above result is not good I think. OpenAI gives a vector distance of 0.1349. Compare with an LLM.

Here are some source models for generating embedings: [hf embeddings](https://huggingface.co/thenlper/gte-large)

A regular LLM like MS/phi-2 is not finetuned to supply embeddings. It also does not have a default padding token id in the tokenizer.. and results in errors.

In [5]:
# model_name = "microsoft/phi-2"
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

In [6]:
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=True, load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Using the "thenlper/gte-large"

In [7]:
gte_large = HuggingFaceEmbeddings(model_name="./data/embeddings/gte-large/", 
                                       model_kwargs={"device": "cuda"}, 
                                       encode_kwargs={"normalize_embeddings": True})

In [12]:
hf_evaluator = load_evaluator("pairwise_embedding_distance", embeddings=gte_large)
hf_evaluator.evaluate_string_pairs(prediction="apple", prediction_b="iphone")

{'score': 0.08266134804374758}

Results in a pretty good score for distance between **apple** and **orange**! Calculated distance as per above is **0.179**!

Remember OpenAI gives a score of **0.135** while the default HuggingFaceEmbeddings gives a score of **0.599**

Now that we know this, we will run the `save_to_chroma()` with another parameter - the function to be used to create the embeddings. Like below:

### Saving the DB to persist on Disk

In [7]:
def save_to_chroma_with_embeddings(chunks: list[Document], embeddings):
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH) ## Recursive delete
    db = Chroma.from_documents(chunks, embeddings, persist_directory=CHROMA_PATH)
    db.persist()

In [8]:
docs = load_documents()
chunks = split_text(docs)
save_to_chroma_with_embeddings(chunks, gte_large)

Split 37 documents into 569 chunks.


### Loading the Database

We need to use the same embedding function that we used to create the DB and the path the DB. Like below:

In [8]:
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=gte_large)

In [18]:
results = db.similarity_search_with_relevance_scores("What can sports betting ads be used for", k=2)
results[1][0].page_content

'Ads that highlight the social aspect of sports betting'

The results of this operation is actually relevant chunks from the vectorised documents that we can use as context to the LLM that can then provide a direct response to the question.

First lets create the template:

In [1]:
PROMPT_TEMPLATE = """
<s>[INST]Answer the question based only on the following context and the chat history:

Context: {context}

History: {history}

Question: {question}[/INST]
"""
prompt = PromptTemplate(template=PROMPT_TEMPLATE, input_variables=["context", "question", "history"])
# context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])

NameError: name 'PromptTemplate' is not defined

### Creating the RetreivalQA Chain

I am not sure how this will pass in the question and context. We dont seem to specify these anywhere..

The Retreival API expects a pipeline however, not the actual model. So we use the HuggingFace pipeline with configuration to return a pipeline object to the RetreivalQA

In [10]:
generation_config = GenerationConfig.from_pretrained(model_name)
generation_config.max_new_tokens = 512
generation_config.temperature = 0.01
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

In [11]:
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)
llm = HuggingFacePipeline(pipeline=text_pipeline)

In [35]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "memory": ConversationBufferMemory(
            memory_key="history",
            input_key="question")},
)


In [37]:
qa_chain.dict

<bound method Chain.dict of RetrievalQA(combine_documents_chain=StuffDocumentsChain(memory=ConversationBufferMemory(input_key='question'), llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'history', 'question'], template='\n<s>[INST]Answer the question based only on the following context and the chat history:\n\nContext: {context}\n\nHistory: {history}\n\nQuestion: {question}[/INST]\n'), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7f38dcc19750>)), document_variable_name='context'), return_source_documents=True, retriever=VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f39100ae4d0>, search_kwargs={'k': 3}))>

In [19]:
sources = "\n".join([doc.metadata["source"] for doc in result["source_documents"]])
result["sources"] = sources

In [20]:
# sources = [doc["metadata"]["source"] for doc in result["source_documents"]]
print("My question:\n{query}\n\nAnswer:\n{result}\n\nSources:\n{sources}".format(**result))

My question:
Could you give me some brief explanation about animated skins? I saw an article about them.

Answer:
Animated skins are a type of advertising format where a background image or design comes to life with motion and animation. They function as a non-intrusive, visually appealing backdrop for ads, allowing them to stand out on websites and apps without disrupting the user experience. Animated skins are fully responsive, adapting to various screen sizes and devices, making them suitable for multi-platform campaigns. Their use has become popular among advertisers seeking to create more engaging and memorable ads. While takeover skins have been in existence for some time, there's been a recent surge in interest due to their effectiveness in capturing audience attention.

Sources:
data/streameye_blog/How_Animated_Skins_Will_Revolutionize_Big_Ad_Campaigns.md
data/streameye_blog/How_Animated_Skins_Will_Revolutionize_Big_Ad_Campaigns.md
data/streameye_blog/How_Animated_Skins_Will_Re

#### Implement History Later

[here](https://stackoverflow.com/questions/76240871/how-do-i-add-memory-to-retrievalqa-from-chain-type-or-how-do-i-add-a-custom-pr) there seems to be a solution

implemented above.

### Checking out Gradio

Try and use Gradio to take in some prompt and output some basic formatted output.

In [39]:
!pip install gradio

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.6/130.6 kB 2.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 73.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.9/996.9 kB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 83.6 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [3]:
import gradio as gr

def chatter(message, history):
    result = qa_chain(message)
    sources = "\n".join([doc.metadata["source"] for doc in result["source_documents"]])
    return f"{result['result']}\n\nSources:{sources}"


In [ ]:
interface = gr.ChatInterface(chatter).launch()

#### Try gradio with threads

In [42]:
from threading import Thread
import gradio as gr

class StatefulThread(Thread):
    def __init__(self, target, args):
        super().__init__(target=target, args=args)
        self._result = None
    def run(self):
        self._result = self._target(self._args)
    def get_result(self):
        return self._result


    
def chatter(message, history):
    PROMPT_TEMPLATE = """
    <s>[INST]Answer the question based only on the following context and the chat history:
    
    Context: {context}
    
    History: {history}
    
    Question: {question}[/INST]
    """
    history_transformer_format = history + [[message, ""]]
    history_str = "".join(["".join(["\n<human>:"+item[0], "\n<bot>:"+item[1]])  #curr_system_message +
                for item in history_transformer_format])
    no_history_prompt = PROMPT_TEMPLATE.replace("{history}", history_str)
    prompt = PromptTemplate(template=no_history_prompt, input_variables=["context", "question"])
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=db.as_retriever(search_kwargs={"k": 3}),
        return_source_documents=True,
        verbose=True,
        chain_type_kwargs={"prompt": prompt, "verbose": True}
    )
    t = StatefulThread(target=qa_chain, args=(message))
    t.start()
    t.join()
    result = t.get_result();
    sources = "\n".join([doc.metadata["source"] for doc in result["source_documents"]])
    return f"{result['result']}\n\nSources:\n{sources}"

interface = gr.ChatInterface(chatter).launch(share=True)

Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://6fdc30d7cba913e4a8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    <s>[INST]Answer the question based only on the following context and the chat history:
    
    Context: Animated skins are increasingly being used by advertisers to make their online campaigns more visually appealing and interactive. These skins are essentially background wallpapers that can be used to display ads without interfering with the site’s original content.

While takeover skins have been around for years, they have recently gained attention due to their effectiveness. In fact, many advertisers are turning to takeover skins to make their ads more engaging and memorable.

Isn’t that just a takeover skin?

Takeover skins are generally static, but it is possible to create dynamic ones with tools like StreamEye. They are generally created to take advantage of full HD resolution (1920x1080) and are designed to fit the dimensions of t

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    <s>[INST]Answer the question based only on the following context and the chat history:
    
    Context: and even sound. And do not worry at all, because HTML banners are compatible with all devices and screen sizes, which means that you can retarget your potential clients on both mobile and desktop.

Have you noticed how attached are people to their phones? Well, the digital marketing industry has not missed it, either. Mobile advertising is on the rise in the last few years and experts believe it could reach a revenue of $384.9 billion by 2023. If you do not want to be left behind, HTML5 banners are the way to go. They are compatible with different devices and screen sizes and are easy to optimize according to your needs. In other words, they will function flawlessly regardless of where

Have all the advantages of the HTML5 banners in mi

...



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    <s>[INST]Answer the question based only on the following context and the chat history:
    
    Context: 3. HTML5 banners are the next iteration of animated online ads. Similar to Flash, they provide interactive and feature-rich options but without the need for extra software or compatibility concerns across different browsers and devices. HTML5 banners have long replaced Flash and are soon to do the same with animated GIFs. They are smaller in size and at the same time you can embed Rich Media features thanks to the fact that HTML5 ads also use CSS and Javascript. This means that you can add

1. Animated GIF banners came after the static ads. GIF stands for Graphic Interchange Format. GIF banners have the .gif extension and can be either static or animated. Because of their nature, they are the middle ground between static and video. Anim

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    <s>[INST]Answer the question based only on the following context and the chat history:
    
    Context: Find out how you can create your own HTML5 banner campaign on Streameye’s platform.

The animated banner evolution: From animated GIFs to HTML5

At Streameye, we make creative work more flexible and agile with a platform that requires less effort from your team. And yes, animated ads production is included and very cost-effective, too. It allows you to become more responsive to market changes and your competitors' actions. With Streameye, you can tweak creatives, update messaging, and change your offers in a matter of minutes, without the help of a developer or even a senior designer. Your custom templates and vast visual library mean

At Streameye, we make creative work more flexible and agile with a platform that requires less effort 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    <s>[INST]Answer the question based only on the following context and the chat history:
    
    Context: Of course, having all of these professionals on your side is crucial for the success of the campaign that eventually you are going to launch. However, maintaining effective communication and workflow can sometimes become challenging and lead to a bit of chaos.

support that will elevate your creative abilities even further. Streameye doesn't just bring you creative templates fit for your brand. Our team will be there to help you get the most out of the platform from the start and stay up to date with the latest digital creative technology. We'll proactively bring you ideas to improve your demand generation results and make use of new creative opportunities. And as much as it’s good if the creative management tool can work according to y

In [13]:
history = []
message = "what are takeover skins"
prompt = "You are an AI assistant that answers questions:"
qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=db.as_retriever(search_kwargs={"k": 3}),
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt, "history": history, "question": message}
    )
class StatefulThread(Thread):
    def __init__(self, target, args):
        super().__init__(target=target, args=args)
        self._result = None
    def run(self):
        self._result = self._target(self._args)
    def get_result(self):
        return self._result

tt = StatefulThread(target=qa_chain, args=("hi there"))
tt.start()
tt.join()
result = tt.get_result();

ValidationError: 1 validation error for LLMChain
prompt
  value is not a valid dict (type=type_error.dict)

{'query': 'hi there',
 'result': "Hi there! Midjourney is an AI art generator that transforms text-based prompts into unique and impressive images. It's an independent tool that can be used to add visual interest to projects like HTML5 banner campaigns. One of its features is animated skins, which can bring banners to life by adding movement and dynamic elements. If you have any questions or would like to learn more, please don't hesitate to book a demo with us.",
 'source_documents': [Document(page_content='We tried out Midjourney.\n\nWe played around with it, unleashed our mutual creativity and, last but not least, we definitely enjoyed the whole process.\n\nWhat is Midjourney and why do I need to care about it?\n\nMidjourney is an independent AI art generator that turns text-based prompts into images. Only with a few words, Midjourney has the capacity to create unique and impressive artworks that will take your breath away.', metadata={'source': 'data/streameye_blog/We_Tried_Midjour